In [0]:
%sql
-- 1. if you dont't have csv file, create external tables from pre-loaded delta tables (use countrie-states-cities-database repo)
CREATE EXTERNAL TABLE main.default.cities
USING CSV
LOCATION 'abfss://con@dlsag83cmland001.dfs.core.windows.net/root/cities'
AS
SELECT * FROM das_azure_postgres_database.public.cities;

In [0]:
%sql
--2. download the csv files
--3. in your cli set the database connection to your postgresql database modifying the following
export PGHOST=aps-ag83-cm-psqlserver-001.postgres.database.azure.com
export PGUSER=delorean
export PGPORT=5432
export PGDATABASE=postgres
export PGPASSWORD=cablemaster\!123 

In [0]:
%sql
--4. craete the schema
psql -f psql_schema.sql

-- 5. import the csv files into the created postgres tables
psql -c "\copy regions FROM 'regions.csv' CSV HEADER NULL 'null'"
psql -c "\copy subregions FROM 'subregions.csv' CSV HEADER NULL 'null'"
psql -c "\copy countries FROM 'countries.csv' CSV HEADER NULL 'null'"
psql -c "\copy states FROM 'states.csv' CSV HEADER NULL 'null'"
psql -c "\copy cities FROM 'cities.csv' CSV HEADER NULL 'null'"

In [0]:
%sql
CREATE OR REPLACE VIEW das_azure_postgres_database.public.world AS
select      -- cities attributes
            c.id                as cities_id,
            c.name              as cities_name,
            c.latitude          as cities_latitude,
            c.longitude         as cities_longitude,
            c.native            as cities_native,
            c.timezone          as cities_timezone,
            c.translations      as cities_translations,
            c.wikiDataId        as cities_wikiDataId,
            -- states attributes
            s.id                as state_id,
            s.name              as state_name,
            s.fips_code         as state_fips_code,
            s.iso2              as state_iso2,
            s.iso3166_2         as state_iso3166_2,
            s.type              as state_type,
            s.level             as state_level,
            s.parent_id         as state_parent_id,
            s.native            as state_native,
            s.latitude          as state_latitude,
            s.longitude         as state_longitude,
            s.timezone          as state_timezone,
            s.translations      as state_translations,
            s.wikiDataId        as state_wikiDataId, 
            -- countries attributes
            cn.id               as country_id,
            cn.name             as country_name,
            cn.iso2             as country_iso2,
            cn.iso3             as country_iso3,
            cn.numeric_code     as country_numeric_code,
            cn.phonecode        as country_phonecode,
            cn.capital          as country_capital,
            cn.currency         as country_currency,
            cn.currency_name    as country_currency_name,
            cn.currency_symbol  as country_currency_symbol,
            cn.tld              as country_tld,
            cn.native           as country_native,
            cn.population       as country_population,
            cn.gdp              as country_gdp,
            cn.nationality      as country_nationality,
            cn.timezones        as country_timezones,
            cn.translations     as country_translations,
            cn.latitude         as country_latitude,
            cn.longitude        as country_longitude,
            cn.emoji            as country_emoji,
            cn.wikiDataId       as country_wikiDataId,
            -- subregions attributes
            sr.id               as subregion_id,
            sr.name             as subregion_name,
            sr.translations     as subregion_translations,
            sr.wikiDataId       as subregion_wikiDataId,
            -- regions attributes
            r.id                as region_id,
            r.name              as region_name,
            r.translations      as region_translations,
            r.wikiDataId        as region_wikiDataId
from        das_azure_postgres_database.public.cities as c
inner join  das_azure_postgres_database.public.states as s
  on        c.state_id = s.id
inner join  das_azure_postgres_database.public.countries as cn
  on        s.country_id = cn.id
inner join  das_azure_postgres_database.public.subregions as sr
  on        cn.subregion_id = sr.id
inner join  das_azure_postgres_database.public.regions as r
  on        sr.region_id = r.id